In [1]:
# Dataset parameters
import os
import sys
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__vsc_ipynb_file__), '../../')))

import torch
from torch import nn
import torchvision
import math
from torchsummary import summary
from super_resolution.srrnet import SRResNet
from super_resolution.drln import DRLN,Drln

In [3]:
dr = DRLN(4)
data = torch.zeros(10, 3, 16, 16)
summary(DRLN(4), (1, 16, 16))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]             640
            Conv2d-2           [-1, 64, 16, 16]          36,928
              ReLU-3           [-1, 64, 16, 16]               0
            Conv2d-4           [-1, 64, 16, 16]          36,928
     ResidualBlock-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]         147,584
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
     ResidualBlock-9          [-1, 128, 16, 16]               0
           Conv2d-10          [-1, 256, 16, 16]         590,080
             ReLU-11          [-1, 256, 16, 16]               0
           Conv2d-12          [-1, 256, 16, 16]         590,080
    ResidualBlock-13          [-1, 256, 16, 16]               0
           Conv2d-14           [-1, 64,

In [5]:
summary(Drln(2,4), (1, 16, 16))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]             640
            Conv2d-2           [-1, 64, 16, 16]          36,928
              ReLU-3           [-1, 64, 16, 16]               0
            Conv2d-4           [-1, 64, 16, 16]          36,928
     ResidualBlock-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]         147,584
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
     ResidualBlock-9          [-1, 128, 16, 16]               0
           Conv2d-10          [-1, 256, 16, 16]         590,080
             ReLU-11          [-1, 256, 16, 16]               0
           Conv2d-12          [-1, 256, 16, 16]         590,080
    ResidualBlock-13          [-1, 256, 16, 16]               0
           Conv2d-14           [-1, 64,

In [7]:
data = torch.randn( 3, 4, 4)

activation = nn.Tanh()
data

tensor([[[-1.7678, -1.9476,  1.0151,  1.5399],
         [ 0.4917,  0.4796,  0.5682, -0.8353],
         [ 0.9308,  0.1327,  1.9139, -0.7093],
         [ 0.7092,  1.0219,  0.9624,  1.4828]],

        [[-0.4525, -0.4096, -1.0911, -0.1063],
         [ 0.5588, -0.8866, -0.6371, -0.6096],
         [-0.5762,  2.2891,  0.4068, -0.4935],
         [ 0.8348,  1.8258,  2.4045,  0.3295]],

        [[-1.3517,  0.1161, -0.5214, -0.0213],
         [ 1.5157,  1.7987,  1.2910,  1.1135],
         [ 1.3541,  0.0864, -1.2122,  1.6288],
         [-0.2906, -1.2289, -1.6006,  3.3147]]])

In [8]:
activation(data)

tensor([[[-0.9434, -0.9601,  0.7679,  0.9121],
         [ 0.4555,  0.4459,  0.5141, -0.6833],
         [ 0.7310,  0.1319,  0.9574, -0.6103],
         [ 0.6101,  0.7707,  0.7454,  0.9020]],

        [[-0.4240, -0.3881, -0.7973, -0.1059],
         [ 0.5071, -0.7097, -0.5629, -0.5438],
         [-0.5199,  0.9797,  0.3857, -0.4570],
         [ 0.6830,  0.9494,  0.9838,  0.3181]],

        [[-0.8745,  0.1155, -0.4788, -0.0213],
         [ 0.9079,  0.9467,  0.8594,  0.8053],
         [ 0.8750,  0.0862, -0.8373,  0.9259],
         [-0.2827, -0.8423, -0.9218,  0.9974]]])